### Install Package

In [ ]:
pip install pyspark

In [ ]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DateType

schema = StructType([
    StructField("product_id",IntegerType(), True),
    StructField("customer_id",StringType(), True),
    StructField("order_date",DateType(), True),
    StructField("location",StringType(), True),
    StructField("source_order",StringType(), True)
])

sales_df = spark.read.format("csv").option("inferschems","true").schema(schema)\
    .load('/kaggle/input/dataset/sales.csv.txt')
sales_df.show()

### Extracting year, quarter and month from order date

In [ ]:
from pyspark.sql.functions import month, year, quarter

sales_df = sales_df.withColumn("order_year",year(sales_df.order_date))
sales_df = sales_df.withColumn("order_quarter",quarter(sales_df.order_date))
sales_df = sales_df.withColumn("order_month",month(sales_df.order_date))
sales_df.show()

### menu dataframe

In [ ]:
schema = StructType([
    StructField("product_id",IntegerType(), True),
    StructField("product_name",StringType(), True),
    StructField("price",StringType(), True)
])

menu_df = spark.read.format("csv").option("inferschems","true").schema(schema)\
    .load('/kaggle/input/dataset/menu.csv.txt')
menu_df.show()

### Total amount spent by each customer

In [ ]:
total_amount_spent1 = sales_df.join(menu_df,'product_id').groupBy("customer_id").agg({'price':'sum'})\
    .orderBy('customer_id')

total_amount_spent1.show()

In [ ]:
import plotly.express as px

pdf = total_amount_spent1.toPandas()

# Create an area plot with Plotly
fig = px.area(pdf, x='customer_id', y='sum(price)', 
              title='Sum of Prices by Customer',
              text='sum(price)')
fig.update_traces(fill='tozeroy')  # Fill area under the line
fig.show()

### Total amount spent by each food category

In [ ]:
total_amount_spent2 = sales_df.join(menu_df,'product_id').groupBy("product_name").agg({'price':'sum'})\
    .orderBy('product_name')

total_amount_spent2.show()

In [ ]:
pdf = total_amount_spent2.toPandas()

# Create an area plot with Plotly
fig = px.bar(pdf, 
             x='sum(price)', 
             y='product_name', 
             title='Sum of Prices by Product', 
             text='sum(price)',
             orientation='h',
             color_discrete_sequence=['green'])
fig.show()

### Total amount of sales in each month

In [ ]:
total_amount_spent3 = sales_df.join(menu_df,'product_id').groupBy('order_month').agg({'price':'sum'})\
    .orderBy('order_month')

total_amount_spent3.show()

In [ ]:
pdf = total_amount_spent3.toPandas()

# Create an area plot with Plotly
fig = px.bar(pdf, 
             x='order_month', 
             y='sum(price)', 
             title='Sum of Prices by Month', 
             text='sum(price)',
             color_discrete_sequence=['brown'])
fig.show()

In [ ]:
yearly_sales = sales_df.join(menu_df,'product_id').groupBy('order_year').agg({'price':'sum'})\
    .orderBy(desc('order_year'))

yearly_sales.show()

In [ ]:
pdf = yearly_sales.toPandas()

# Create an area plot with Plotly
fig = px.line(pdf, 
              x='order_year', 
              y='sum(price)', 
              title='Yearly Sales',
              markers=True)
fig.show()

In [ ]:
quarterly_sales = sales_df.join(menu_df,'product_id').groupBy('order_quarter').agg({'price':'sum'})\
    .orderBy(desc('order_quarter'))

quarterly_sales.show()

In [ ]:
pdf = quarterly_sales.toPandas()

fig = px.pie(pdf,
            names='order_quarter',
            values='sum(price)',
            title='Quarterly Sales',
            hole=0.4
            )
fig.show()

### Howmany times each product has been purchased

In [ ]:
from pyspark.sql.functions import count

most_df = sales_df.join(menu_df,'product_id').groupBy('product_id','product_name')\
    .agg(count('product_id').alias('product_count')).orderBy('product_count',ascending=0)\
    .drop('product_id')
most_df.show()

In [ ]:
pdf = most_df.toPandas()

fig = px.funnel(pdf,
                x='product_count',
                y='product_name',
                title='Product Sales Funnel Visual'
)

fig.show()

### Top 5 ordered items

In [ ]:
most_df = sales_df.join(menu_df,'product_id').groupBy('product_id','product_name')\
    .agg(count('product_id').alias('product_count')).orderBy('product_count',ascending=0)\
    .drop('product_id').limit(5)
most_df.show()

### Top ordered Item!

In [ ]:
most_df = sales_df.join(menu_df,'product_id').groupBy('product_id','product_name')\
    .agg(count('product_id').alias('product_count')).orderBy('product_count',ascending=0)\
    .drop('product_id').limit(1)
most_df.show()

### Frequency of customer visit to restaurant

In [ ]:
from pyspark.sql.functions import countDistinct

cust_freq = sales_df.filter(sales_df.source_order=='Restaurant').groupBy('customer_id').agg(countDistinct('order_date'))
cust_freq.show()

In [ ]:
pdf = cust_freq.toPandas()

# Create an area plot with Plotly
fig = px.bar(pdf, 
             x='count(DISTINCT order_date)', 
             y='customer_id', 
             title='Frequency of Restaurant Customers', 
             text='count(DISTINCT order_date)',
             orientation='h',
             color_discrete_sequence=['orange'])
fig.show()

### Total sale by each country

In [ ]:
sales_by_country = sales_df.join(menu_df,'product_id').groupBy("location").agg({'price':'sum'})

sales_by_country.show()

In [ ]:
pdf = sales_by_country.toPandas()

# Create an area plot with Plotly
fig = px.bar(pdf, 
             x='sum(price)', 
             y='location', 
             title='Total sales by each country', 
             text='sum(price)',
             orientation='h',
             color_discrete_sequence=['teal'])
fig.show()

### Total sales by order source

In [ ]:
sales_by_order_source = sales_df.join(menu_df,'product_id').groupBy("source_order").agg({'price':'sum'})

sales_by_order_source.show()

In [ ]:
pdf = sales_by_order_source.toPandas()

fig = px.pie(pdf,
            names='source_order',
            values='sum(price)',
            title='Total sales by order source',
            hole=0.3,
            color_discrete_sequence=px.colors.qualitative.Pastel1
            )
fig.show()